In [51]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from utils.quack import Quack
from sim_utils.modeling import update_config, maskedModel, ResBlock, FocalLoss, FocalLossSoft, adjacent_label_smoothing #noqa
torch.cuda.is_available()

True

In [54]:
kickers = Quack.query("""
	with raw as (
select
	pbp.season,
	pbp.week,
	pbp.game_id,
	kick_distance,
	yardline_100,
	return_yards,
	yardline_100 + kick_distance - return_yards as ending_yardline,
	kicker_player_id,
	kicker_player_name,
	kickoff_returner_player_name,
	kickoff_returner_player_id,
	case when wind is null then 3 else wind end as wind,
	case when temp is null then 65 else temp end as temp,
	case when roof == 'dome' or roof == 'closed' then 1 else 0 end as is_indoors
	from pbp 
	)
	select kicker_player_id,
					  kicker_player_name,
					  avg(kick_distance),
					  avg(yardline_100 + kick_distance) as avg_landing_spot 
					  from raw
					  group by all
""")
kickers

IOException: IO Error: File is already open in 
C:\Users\Sharedw\AppData\Local\Programs\Python\Python312\python.exe (PID 63692)

In [48]:
Quack.query("""
select 
"desc",
				pbp.season,
	pbp.week,
	pbp.game_id,
	kick_distance,
	yardline_100,
	return_yards,
	yardline_100 + kick_distance - return_yards as ending_yardline,
	kicker_player_id,
	kicker_player_name,
	kickoff_returner_player_name,
	kickoff_returner_player_id,
			play_type,
			yards_gained
			 from pbp 
			where game_id = '2000_01_ARI_NYG'
			
			""")

,desc,season,week,game_id,kick_distance,yardline_100,return_yards,ending_yardline,kicker_player_id,kicker_player_name,kickoff_returner_player_name,kickoff_returner_player_id,play_type,yards_gained
0,B.Daluiso kicks 72 yards from NYG 30 to ARI -2...,2000,1,2000_01_ARI_NYG,NaN,30.0,0.0,NaN,None,None,None,None,no_play,0.0
1,B.Daluiso kicks 69 yards from NYG 25 to ARI 6....,2000,1,2000_01_ARI_NYG,69.0,25.0,29.0,65.0,00-0003841,B.Daluiso,M.Jenkins,00-0008357,kickoff,0.0
2,(15:00) T.Jones left end to ARI 37 for 2 yards...,2000,1,2000_01_ARI_NYG,NaN,65.0,0.0,NaN,None,None,None,None,run,2.0
3,(14:04) J.Plummer pass incomplete to T.Jones.,2000,1,2000_01_ARI_NYG,NaN,63.0,0.0,NaN,None,None,None,None,pass,0.0
4,(13:57) (Shotgun) J.Plummer pass incomplete to...,2000,1,2000_01_ARI_NYG,NaN,63.0,0.0,NaN,None,None,None,None,pass,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,(:39) (Shotgun) J.Plummer pass to D.Boston for...,2000,1,2000_01_ARI_NYG,NaN,25.0,0.0,NaN,None,None,None,None,pass,25.0
178,TWO-POINT CONVERSION ATTEMPT. J.Plummer pass t...,2000,1,2000_01_ARI_NYG,NaN,2.0,0.0,NaN,None,None,None,None,pass,0.0
179,(Onside Kick formation) C.Blanchard kicks 23 y...,2000,1,2000_01_ARI_NYG,23.0,30.0,0.0,53.0,00-0001343,C.Blanchard,None,None,kickoff,0.0
180,(:32) K.Collins to NYG 46 for -1 yards.,2000,1,2000_01_ARI_NYG,NaN,53.0,0.0,NaN,None,None,None,None,run,-1.0


In [ ]:
kicks = Quack.query("""
select
	pbp.season,
	pbp.week,
	pbp.game_id,
	kick_distance,
	yardline_100,
	return_yards,
	yardline_100 + kick_distance - return_yards as ending_yardline,
	kicker_player_id,
	kicker_player_name,
	kickoff_returner_player_name,
	kickoff_returner_player_id
	from pbp 
	--join team_feats as t 
	--on pbp.posteam = t.team
	--and pbp.game_id = t.game_id
	--join opp_feats as o
	--on pbp.defteam = o.opponent_team
	--and pbp.game_id = o.game_id
	--join player_weekly_agg as r
	--on r.game_id = pbp.game_id
	--and pbp.rusher_player_id = r.gsis_id
	--and r.team = pbp.posteam
	where pbp.play_type = 'kickoff'
	and season = 2025
	order by return_yards desc
""")


kicks.head(20)

,season,week,game_id,kick_distance,yardline_100,return_yards,ending_yardline,kicker_player_id,kicker_player_name,kickoff_returner_player_name,kickoff_returner_player_id
0,2025,10,2025_10_CLE_NYJ,64.0,35.0,99.0,0.0,00-0038428,A.Szmyt,K.Nwangwu,00-0036842
1,2025,9,2025_09_CHI_CIN,63.0,35.0,98.0,0.0,00-0031203,C.Santos,C.Jones,00-0038576
2,2025,11,2025_11_SF_ARI,64.0,35.0,98.0,1.0,00-0038567,C.Ryland,S.Moore,00-0038090
3,2025,12,2025_12_BUF_HOU,62.0,35.0,97.0,0.0,00-0032726,K.Fairbairn,R.Davis,00-0039875
4,2025,2,2025_02_NE_MIA,55.0,35.0,90.0,0.0,00-0036816,R.Patterson,A.Gibson,00-0036328
5,2025,7,2025_07_IND_LAC,60.0,35.0,81.0,14.0,00-0037224,C.Dicker,A.Abdullah,00-0032104
6,2025,6,2025_06_DEN_NYJ,61.0,35.0,72.0,24.0,00-0032569,W.Lutz,K.Nwangwu,00-0036842
7,2025,1,2025_01_TEN_DEN,60.0,35.0,71.0,24.0,00-0032569,W.Lutz,C.Dike,00-0040705
8,2025,3,2025_03_LV_WAS,61.0,35.0,69.0,27.0,00-0034161,D.Carlson,D.Samuel,00-0035719
9,2025,5,2025_05_TEN_ARI,64.0,35.0,65.0,34.0,00-0038567,C.Ryland,C.Dike,00-0040705
